In [114]:
#%matplotlib inline
import sys
import matplotlib    
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.ticker
import csv
import math
import numpy as np
from stats import CSVStats


# Our CSVs have a space after the comma, so we need a new 'dialect', here
# called 'deploy'
csv.register_dialect('deploy', delimiter=',', doublequote=False, quotechar='',
                     lineterminator='\n', escapechar='',
                     quoting=csv.QUOTE_NONE, skipinitialspace=True)

color1_light = 'lightgreen'
color1_dark = 'green'
color2_light = 'lightblue'
color2_dark = 'blue'
color3_light = 'yellow'
color3_dark = 'brown'
color4_light = 'pink'
color4_dark = 'red'

# Holds different methods to plot on a given graph
class MPlot:
    show_fig = True
    pngname = ""
    plt = None

    def __init__(self):
        vers = matplotlib.__version__
        if vers != "1.5.1":
            print "\nWrong matlib-version " + vers + ", please install 1.5.1"
            print "http://matplotlib.org/faq/installing_faq.html\n"
            print "Or try the following\nsudo easy_install \"matplotlib == 1.5.1\"\n"
            exit(1)
        self.plt = plt
        self.resetMinMax()
        
    # Will put (x|y)(min|max) to default values
    def resetMinMax(self):
        self.ymin = -1
        self.ymax = 0
        self.xmin = -1
        self.xmax = 0

    # Updates the xmin and xmax with the given values on the x-axis
    def updateX(self, *values):
        for v in values:
            if self.xmin == -1:
                self.xmin = min(v)
            else:
                self.xmin = min(self.xmin, min(v))
            self.xmax = max(self.xmax, max(v))

    # Updates the xmin and xmax with the given values on the y-axis
    def updateY(self, *values):
        for v in values:
            if self.ymin == -1:
                self.ymin = min(v)
            else:
                self.ymin = min(self.ymin, min(v))
            self.ymax = max(self.ymax, max(v))
            

            
    # Takes one x and y1, y2 to stack y2 on top of y1. Does all the
    # calculation necessary to sum up everything
    def plotStackedBars(self, stats, values1, values2, label1, label2, color1,
                        color2, ymin=None,
                        delta_x=0, values=0):
        val1 = stats.get_values(values1)
        val2 = stats.get_values(values2)
        x = val1.x
        y1 = val1.avg
        y2 = val2.avg
        if values > 0:
            y1 = y1[0:values - 1]
            y2 = y2[0:values - 1]
            x = x[0:values - 1]
        width = [(t * 0.125 + delta_x * t * 0.018)*5 for t in x]

        zero = [min(y1) for t in y1]
        xd = [t[0] + delta_x * t[1] for t in zip(stats.x, width)]
        y12 = [sum(t) for t in zip(y1, y2)]
        
        plt.bar(xd, y12, width, color=color1, bottom=y1, zorder=3, label=label1, align='center')
        plt.bar(xd, y1, width, color=color2, bottom=zero, zorder=3,
                label=label2, align='center')

    # Puts the most used arguments for starting a plot with
    # LogLog by default.
    def plotPrepareLogLog(self, logx=2, logy=2):
        plt.clf()
        plt.Figure()
        plt.ylabel('CPU-time per node')
        plt.xlabel('Number of nodes')
        if logx > 0:
            plt.xscale(u'log', basex=logx)
        if logy > 0:
            plt.yscale(u'log', basey=logy)

        ax = plt.axes()
        ax.yaxis.grid(color='gray', linestyle='dashed', zorder=0)
        ax.xaxis.set_major_formatter(
            matplotlib.ticker.ScalarFormatter(useOffset=False))
        ax.xaxis.set_zorder(5)
        sf = matplotlib.ticker.ScalarFormatter()
        sf.set_powerlimits((-10, 10))
        sf.set_scientific(False)
        # ax.yaxis.set_major_formatter(sf)
        # ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%2.2e'))
        if logy == 2:
            ax.yaxis.set_major_formatter(
                matplotlib.ticker.FormatStrFormatter('%2.2f'))

    # Ends the plot and takes an extension for saving the png. If
    # show_fig is True, it will show the window instead.
    def plotEnd(self):
        if self.show_fig:
            print "Showing plot"
            plt.show()
        else:
            print "Saving to", self.pngname
            for ext in ['png', 'eps']:
                plt.savefig(self.pngname + "." + ext, bbox_inches='tight')

        self.resetMinMax()

    # If we want to remove a poly
    def delete_poly(self, poly):
        self.poly.remove()

    # For removing a line
    def delete_line(self, line):
        self.line[0].remove()
        if len(self.line) > 1:
            for i in range(1, 3):
                for l in self.line[i]:
                    l.remove()

## Number of packages in the repository

In [115]:
mplot = MPlot()

def plot_save(file):
    mplot.pngname = file
    mplot.show_fig = False

stats = "verify_proofs_wall" # client_receive_latest_release_wall client_bw_debianupdate_rx verify_proofs_wall
argn = len(sys.argv)
if argn < 2:
    print "Syntax is:"
    print "plot.py file.csv [stat]"
    print "where stat is one of {round_wall, round_system, round_user}"
    sys.exit(1)

if argn > 2:
    stats = "verify_proofs_wall" # client_receive_latest_release_wall  client_bw_debianupdate_rx 
    
data = CSVStats("number_of_packages_in_repo.csv", "numberofpackagesinrepo")
mplot.plotPrepareLogLog(10, 10)
        
na = mplot.plotStackedBars(data, "verify_signature_wall", "verify_proofs_wall", "CPU time to verify the root signature",
                    "CPU time to verify the proofs", color1_dark, color2_dark)
#na = mplot.plotMMA(data, stats, color1_light, 0,
#                    dict(label="Debian APT bandwidth for testing", linestyle='-', marker='s', color=color2_dark, zorder=3))

plt.legend(loc='upper left')
plt.ylabel('Time for verification')
plt.xlabel('Number of packages in the repository.')
mplot.plotEnd()

Showing plot


## Cothority metrics

In [117]:
mplot = MPlot()

def plot_save(file):
    mplot.pngname = file
    mplot.show_fig = False

stats = "verify_proofs_wall"
 
data = CSVStats("debianupdate_1_client_testing_number_of_packages_in_repo.csv", "numberofpackagesinrepo")
mplot.plotPrepareLogLog(10, 10)
        
val1 = data.get_values("cothority_signing_wall")
val2 = data.get_values("cothority_verify_proofs_wall")
val3 = data.get_values("cothority_indiv_sign_verif_wall")
x = val1.x
y1 = val1.avg
y2 = val2.avg
y3 = val3.avg
delta_x = 0

width = [(t * 0.125 + delta_x * t * 0.018)*7 for t in x]
xd = [t[0] + delta_x * t[1] for t in zip(data.x, width)]

y12 = [sum(t) for t in zip(y1, y2)]
y123 = [sum(t) for t in zip(y1, y2, y3)]

zero = [min(y1+y2+y3) for t in y3]

plt.bar(xd, y1, width, color=color1_dark, bottom=zero, zorder=3, 
        label="Sign the root", align='center')
plt.bar(xd, y12, width, color=color3_dark, bottom=y1, zorder=3,
        label="Verify individual packages signatures from each Skipchains", align='center')
plt.bar(xd, y123, width, color=color2_dark, bottom=y12, zorder=3,
        label="Build Merkle-Tree and verify the proofs", align='center')
plt.xticks(xd, [1, 10, 100, 1000, 10000, 52666])
        
plt.legend(loc='upper left')
plt.ylabel('CPU Time for verification')
plt.xlabel('Number of packages in the repository.')
mplot.plotEnd()

Showing plot


## Client Bandwidth

In [116]:
mplot = MPlot()

def plot_save(file):
    mplot.pngname = file
    mplot.show_fig = False

stats = "deb_get" 
data = CSVStats("debianupdate_test.csv", "numberofpackagesinrepo")
mplot.plotPrepareLogLog(10, 10)
na = mplot.plotMMA(data, stats, color1_light, 0,
                    dict(label="Debian APT bandwidth for testing", linestyle='-', marker='s', color=color2_dark, zorder=3))#, align='center'))

na = mplot.plotMMAcumul(data, "deb_get", "client_bw_debianupdate_rx", color1_light, 0,
                    dict(label="APT + Additional Bandwidth for\nDebian Update Service", linestyle='-', marker='s', color=color3_dark, zorder=3))
plt.legend(loc='upper left')
plt.ylabel('Bandwidth [bytes]')
plt.xlabel('Number of packages in the repository.')
mplot.plotEnd()

AttributeError: MPlot instance has no attribute 'plotMMA'

# OLD

In [107]:
# Plots the graph of one of the test-runs
# It takes the CSV-file as argument and shows the plot
# of the times used for each round

import sys

try:
    import matplotlib
except ImportError:
    print "Please install Matplotlib:"
    print "\nsudo easy_install \"matplotlib == 1.5.1\"\n"
    sys.exit(1)

matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.ticker
import csv
import math
import numpy as np

# Our CSVs have a space after the comma, so we need a new 'dialect', here
# called 'deploy'
csv.register_dialect('deploy', delimiter=',', doublequote=False, quotechar='',
                     lineterminator='\n', escapechar='',
                     quoting=csv.QUOTE_NONE, skipinitialspace=True)


# Holds different methods to plot on a given graph
class MPlot:
    show_fig = True
    pngname = ""
    plt = None

    def __init__(self):
        vers = matplotlib.__version__
        if vers != "1.5.1":
            print "\nWrong matlib-version " + vers + ", please install 1.5.1"
            print "http://matplotlib.org/faq/installing_faq.html\n"
            print "Or try the following\nsudo easy_install \"matplotlib == 1.5.1\"\n"
            exit(1)
        self.plt = plt
        self.resetMinMax()

    # Will put (x|y)(min|max) to default values
    def resetMinMax(self):
        self.ymin = -1
        self.ymax = 0
        self.xmin = -1
        self.xmax = 0

    # Updates the xmin and xmax with the given values on the x-axis
    def updateX(self, *values):
        for v in values:
            if self.xmin == -1:
                self.xmin = min(v)
            else:
                self.xmin = min(self.xmin, min(v))
            self.xmax = max(self.xmax, max(v))

    # Updates the xmin and xmax with the given values on the y-axis
    def updateY(self, *values):
        for v in values:
            if self.ymin == -1:
                self.ymin = min(v)
            else:
                self.ymin = min(self.ymin, min(v))
            self.ymax = max(self.ymax, max(v))

    # Plots the Minimum, Maximum, Average on the same plot.
    def plotMMA(self, stats, values, plot_color, plot_z, args):
        val = stats.get_values(values)
        plt.plot(val.x, val.avg, **args)
        self.plotFilledLegend(val, "min-max", plot_color, z=plot_z)
        return val
    
    # Plots the Minimum, Maximum, Average on the same plot.
    def plotMMAcumul(self, stats, values, cumul, plot_color, plot_z, args):
        val = stats.get_values(values)
        val2 = stats.get_values(cumul)
        cml = [x + y for x, y in zip(val.avg, val2.avg)]
        print cml
        plt.plot(val.x, cml, **args)
        self.plotFilledLegend(val, "min-max", plot_color, z=plot_z)
        return val

    # Adds a fill_between and the corresponding 'empty' plot to show up in
    # the legend
    def plotFilledLegend(self, stats, label, color, z=None):
        x, y1, y2 = stats.x, stats.min, stats.max
        if z:
            fb = plt.fill_between(x, y1, y2, facecolor=color, edgecolor='white',
                                  zorder=z)
        else:
            fb = plt.fill_between(x, y1, y2, facecolor=color, edgecolor='white',
                                  zorder=3)

        self.updateX(x)
        self.updateY(y1, y2)
        # plt.plot([], [], '-', label=label, color=color, linewidth=10)

    # Takes one x and y1, y2 to stack y2 on top of y1. Does all the
    # calculation necessary to sum up everything
    def plotStacked(self, stats, col1, col2, label1, label2, color1, color2,
                    ymin=None, values=0):
        #stats.reset_min_max()
        y1 = stats.get_values(col1)
        y2 = stats.get_values(col2)
        #if values > 0:
        #    y1 = y1[0:values - 1]
        #    y2 = y2[0:values - 1]
        #if ymin == None:
        #    ymin = min(min(y1), min(y2))
        val = stats.get_values(values)
        ymins = [ymin] * len(val.x)
        ysum = [sum(t) for t in zip(y1, y2)]


        self.plotFilledLegend(val.x, y1, ysum, label2, color2)
        self.plotFilledLegend(val.x, ymins, y1, label1, color1)

    # Takes one x and y1, y2 to stack y2 on top of y1. Does all the
    # calculation necessary to sum up everything
    def plotStackedBars(self, stats, values1, values2, label1, label2, color1,
                        color2, ymin=None,
                        delta_x=0, values=0):
        val1 = stats.get_values(values1)
        val2 = stats.get_values(values2)
        x = val1.x
        y1 = val1.avg
        y2 = val2.avg
        if values > 0:
            y1 = y1[0:values - 1]
            y2 = y2[0:values - 1]
            x = x[0:values - 1]
        width = [(t * 0.125 + delta_x * t * 0.018)*5 for t in x]

        zero = [min(y1) for t in y1]
        xd = [t[0] + delta_x * t[1] for t in zip(stats.x, width)]
        y12 = [sum(t) for t in zip(y1, y2)]
        
        plt.bar(xd, y12, width, color=color1, bottom=y1, zorder=3, label=label1, align='center')
        plt.bar(xd, y1, width, color=color2, bottom=zero, zorder=3,
                label=label2, align='center')

    # Takes one x and y1, y2 to stack y2 on top of y1. Does all the
    # calculation necessary to sum up everything
    def plotStackedBarsHatched(self, stats, values1, values2, label, color,
                               ymin=None,
                               limit_values=None, delta_x=0):
        val1 = stats.get_values(values1)
        val2 = stats.get_values(values2)
        x = val1.x[::2]
        y1 = val1.avg[::2]
        y2 = val2.avg[::2]
        if limit_values != None:
            x = x[0:limit_values]
            y1 = y1[0:limit_values]
            y2 = y2[0:limit_values]
        w = 1. / 2.5
        l = 2

        xd = [t * math.pow(l, delta_x * w) for t in x]
        width = [t * ( math.pow(l, w) - 1) for t in xd]
        y12 = [sum(t) for t in zip(y1, y2)]
        plt.bar(xd, y12, width, color=color, bottom=y1, zorder=3, hatch='//')
        return plt.bar(xd, y1, width, color=color, bottom=ymin, zorder=3,
                       label=label), val1, val2

    # Puts the most used arguments for starting a plot with
    # LogLog by default.
    def plotPrepareLogLog(self, logx=2, logy=2):
        plt.clf()
        plt.Figure()
        plt.ylabel('CPU-time per node')
        plt.xlabel('Number of nodes')
        if logx > 0:
            plt.xscale(u'log', basex=logx)
        if logy > 0:
            plt.yscale(u'log', basey=logy)

        ax = plt.axes()
        ax.yaxis.grid(color='gray', linestyle='dashed', zorder=0)
        ax.xaxis.set_major_formatter(
            matplotlib.ticker.ScalarFormatter(useOffset=False))
        ax.xaxis.set_zorder(5)
        sf = matplotlib.ticker.ScalarFormatter()
        sf.set_powerlimits((-10, 10))
        sf.set_scientific(False)
        # ax.yaxis.set_major_formatter(sf)
        # ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%2.2e'))
        if logy == 2:
            ax.yaxis.set_major_formatter(
                matplotlib.ticker.FormatStrFormatter('%2.2f'))

    # Ends the plot and takes an extension for saving the png. If
    # show_fig is True, it will show the window instead.
    def plotEnd(self):
        if self.show_fig:
            print "Showing plot"
            plt.show()
        else:
            print "Saving to", self.pngname
            for ext in ['png', 'eps']:
                plt.savefig(self.pngname + "." + ext, bbox_inches='tight')

        self.resetMinMax()

    # Draws an arrow for out-of-bound data
    def arrow(self, text, x, top, dist, color):
        plt.annotate(text, xy=(x, top), xytext=(x, top - dist),
                     arrowprops=dict(facecolor=color, headlength=5, width=6,
                                     headwidth=10, edgecolor='white'),
                     horizontalalignment='center', )

    # If we want to remove a poly
    def delete_poly(self, poly):
        self.poly.remove()

    # For removing a line
    def delete_line(self, line):
        self.line[0].remove()
        if len(self.line) > 1:
            for i in range(1, 3):
                for l in self.line[i]:
                    l.remove()


In [ ]:
#!/usr/bin/env python
# Simple plot-binary for showing one graph. Syntax is:
#
#   plot.py file.csv [stat]
#
# where file.csv is the file to print and stat is an optional argument taking
# one of {round_wall, round_system, round_user} or another statistic present
# in the csv-file.

import sys
from stats import CSVStats
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Colors for the Cothority
color1_light = 'lightgreen'
color1_dark = 'green'
color2_light = 'lightblue'
color2_dark = 'blue'
color3_light = 'yellow'
color3_dark = 'brown'
color4_light = 'pink'
color4_dark = 'red'
mplot = MPlot()

def plot_save(file):
    mplot.pngname = file
    mplot.show_fig = False

stats = "verify_proofs_wall" # client_receive_latest_release_wall client_bw_debianupdate_rx verify_proofs_wall
argn = len(sys.argv)
if argn < 2:
    print "Syntax is:"
    print "plot.py file.csv [stat]"
    print "where stat is one of {round_wall, round_system, round_user}"
    sys.exit(1)

if argn > 2:
    stats = "verify_proofs_wall" # client_receive_latest_release_wall  client_bw_debianupdate_rx 
    
data = CSVStats("number_of_packages_in_repo.csv", "numberofpackagesinrepo")
mplot.plotPrepareLogLog(10, 10)
        
na = mplot.plotStackedBars(data, "verify_signature_wall", "verify_proofs_wall", "CPU time to verify the root signature",
                    "CPU time to verify the proofs", color1_dark, color2_dark)
#na = mplot.plotMMA(data, stats, color1_light, 0,
#                    dict(label="Debian APT bandwidth for testing", linestyle='-', marker='s', color=color2_dark, zorder=3))

plt.legend(loc='upper left')
plt.ylabel('Time for verification')
plt.xlabel('Number of packages in the repository.')
mplot.plotEnd()

In [103]:
#!/usr/bin/env python
# Simple plot-binary for showing one graph. Syntax is:
#
#   plot.py file.csv [stat]
#
# where file.csv is the file to print and stat is an optional argument taking
# one of {round_wall, round_system, round_user} or another statistic present
# in the csv-file.

import sys
from stats import CSVStats
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Colors for the Cothority
color1_light = 'lightgreen'
color1_dark = 'green'
color2_light = 'lightblue'
color2_dark = 'blue'
color3_light = 'yellow'
color3_dark = 'brown'
color4_light = 'pink'
color4_dark = 'red'
mplot = MPlot()

def plot_save(file):
    mplot.pngname = file
    mplot.show_fig = False

stats = "deb_get" # client_receive_latest_release_wall client_bw_debianupdate_rx verify_proofs_wall
argn = len(sys.argv)
if argn < 2:
    print "Syntax is:"
    print "plot.py file.csv [stat]"
    print "where stat is one of {round_wall, round_system, round_user}"
    sys.exit(1)

if argn > 2:
    stats = "deb_get" # client_receive_latest_release_wall  client_bw_debianupdate_rx verify_proofs_wall
    
data = CSVStats("debianupdate_test.csv", "numberofpackagesinrepo")
mplot.plotPrepareLogLog(10, 10)
na = mplot.plotMMA(data, stats, color1_light, 0,
                    dict(label="Debian APT bandwidth for testing", linestyle='-', marker='s', color=color2_dark, zorder=3))#, align='center'))

na = mplot.plotMMAcumul(data, "deb_get", "client_bw_debianupdate_rx", color1_light, 0,
                    dict(label="APT + Additional Bandwidth for\nDebian Update Service", linestyle='-', marker='s', color=color3_dark, zorder=3))
plt.legend(loc='upper left')
plt.ylabel('Bandwidth [bytes]')
plt.xlabel('Number of packages in the repository.')
mplot.plotEnd()

[1387.229049260085, 5697.2904926008505, 58275.9049260085, 680128.049260085, 8149481.49260085, 45890056.0]
Showing plot


In [ ]:
#!/usr/bin/env python
# Simple plot-binary for showing one graph. Syntax is:
#
#   plot.py file.csv [stat]
#
# where file.csv is the file to print and stat is an optional argument taking
# one of {round_wall, round_system, round_user} or another statistic present
# in the csv-file.

import sys
from stats import CSVStats
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Colors for the Cothority
color1_light = 'lightgreen'
color1_dark = 'green'
color2_light = 'lightblue'
color2_dark = 'blue'
color3_light = 'yellow'
color3_dark = 'brown'
color4_light = 'pink'
color4_dark = 'red'
mplot = MPlot()

def plot_save(file):
    mplot.pngname = file
    mplot.show_fig = False

stats = "verify_proofs_wall" # client_receive_latest_release_wall client_bw_debianupdate_rx verify_proofs_wall
argn = len(sys.argv)
if argn < 2:
    print "Syntax is:"
    print "plot.py file.csv [stat]"
    print "where stat is one of {round_wall, round_system, round_user}"
    sys.exit(1)

if argn > 2:
    stats = "verify_proofs_wall" # client_receive_latest_release_wall  client_bw_debianupdate_rx 
    
data = CSVStats("debianupdate_test.csv", "numberofpackagesinrepo")
mplot.plotPrepareLogLog(10, 10)

    # Takes one x and y1, y2 to stack y2 on top of y1. Does all the
    # calculation necessary to sum up everything
    def plotStackedBars(self, stats, values1, values2, label1, label2, color1,
                        color2, ymin=None,
                        delta_x=0, values=0):
        
na = plotStackedBars(data, "verify_signature_wall", "verify_proofs_wall", "CPU time to verify signature", )
na = mplot.plotMMA(data, stats, color1_light, 0,
                    dict(label="Debian APT bandwidth for testing", linestyle='-', marker='s', color=color2_dark, zorder=3))

plt.legend(loc='upper left')
plt.ylabel('Bandwidth [bytes]')
plt.xlabel('Number of packages in the repository.')
mplot.plotEnd()

In [ ]:
#!/usr/bin/env python
# Simple plot-binary for showing one graph. Syntax is:
#
#   plot.py file.csv [stat]
#
# where file.csv is the file to print and stat is an optional argument taking
# one of {round_wall, round_system, round_user} or another statistic present
# in the csv-file.

import sys
from stats import CSVStats
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Colors for the Cothority
color1_light = 'lightgreen'
color1_dark = 'green'
color2_light = 'lightblue'
color2_dark = 'blue'
color3_dark = 'brown'
color4_light = 'pink'
color4_dark = 'red'
mplot = MPlot()

def plot_save(file):
    mplot.pngname = file
    mplot.show_fig = False

stats = "verify_proofs_wall" # client_receive_latest_release_wall client_bw_debianupdate_rx verify_proofs_wall
argn = len(sys.argv)
if argn < 2:
    print "Syntax is:"
    print "plot.py file.csv [stat]"
    print "where stat is one of {round_wall, round_system, round_user}"
    sys.exit(1)

if argn > 2:
    stats = "verify_proofs_wall" # client_receive_latest_release_wall  client_bw_debianupdate_rx 
    
data = CSVStats("number_of_installed_packages.csv", "numberofinstalledpackages")
mplot.plotPrepareLogLog(10, 10)
        
na = mplot.plotStackedBars(data, "verify_signature_wall", "verify_proofs_wall", "CPU time to verify the root signature",
                    "CPU time to verify the proofs", color1_dark, color2_dark)
#na = mplot.plotMMA(data, stats, color1_light, 0,
#                    dict(label="Debian APT bandwidth for testing", linestyle='-', marker='s', color=color2_dark, zorder=3))

plt.legend(loc='upper left')
plt.ylabel('Time for verification')
plt.xlabel('Number of installed packages on client.')
mplot.plotEnd()